Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = ""
COLLABORATORS = ""

---

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [2]:
NAME = ""
COLLABORATORS = ""

---

In [3]:
!pip install geojson
!pip install shapely
!pip install PyShp
!pip install networkx


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

### Helper functions

In [5]:
def isDistrictContiguous(district_num, assignment, contiguity_list, print_isolates=False, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    if(print_isolates):
        print(list(nx.isolates(district_graph)))
    return nx.is_connected(district_graph)


def get_cut_edges(current_map_dict, adj_list_dict):
    cut_edges = 0
    for precinct, neighbors in adj_list_dict.items():
        current_district = current_map_dict[precinct]
        for neighbor in neighbors:
            if neighbor in current_map_dict and current_map_dict[neighbor] != current_district:
                cut_edges += 1
    return cut_edges / 2

In [6]:
def getDistrictPopulations(assignment,data_file, num_district):
    population = {}
    for i in range (1,num_district+1):
        population[i] = data_file[data_file['GEOID20'].isin(assignment[assignment['District']==i]['GEOID20'])]['Total_2020_Total'].sum()
    return population

In [7]:
def getDistrictShape(district_id, assignment, boundaries):
    list_precincts = assignment[assignment['District']==district_id]['GEOID20']
    precinct_shapes = []
    for i in list_precincts:
        if shape(boundaries[i]).geom_type == 'Polygon':
            precinct_shapes.append(Polygon(shape(boundaries[i])))
        elif shape(boundaries[i]).geom_type == 'MultiPolygon':
            precinct_shapes.append(MultiPolygon(shape(boundaries[i])))      
    district_shape = shapely.ops.unary_union(precinct_shapes)
    #print(district_shape)
    return district_shape

In [8]:
def pp_compactness(geom): # Polsby-Popper
    p = geom.length
    a = geom.area    
    return (4*np.pi*a)/(p*p)

def box_reock_compactness(geom): # Reock on a rectangle bounding box
    a = geom.area 
    bb = geom.bounds # bounds gives you the minimum bounding box (rectangle)
    bba = abs(bb[0]-bb[2])*abs(bb[1]-bb[3])
    return a/bba

# This Notebook will help you get started on NJ
The data is in Canvas, you should upload it to your Google Drive first (if using Colab), or local filesystem (if using Jupyter).

### This is the current assignment of precinct to congressional districts (12 of them for NJ)
Note that the map shown in DRA is slightly different. This is because some precincts are split in the real assignment, and some additional precinct are created to handle special situations such as prisoners and overseas citizens. You can ignore this for the class project and just use the data and functions provided.

In [9]:
nj_current_assignment = pd.read_csv('Map_Data/precinct-assignments-congress-nj.csv')
nj_current_assignment

FileNotFoundError: [Errno 2] No such file or directory: 'Map_Data/precinct-assignments-congress-nj.csv'

### This is the current demographic and voter data
The data has a lot of attributes that lists voters of different demographics and parties in different elections. You can look at the data Dictionary on Canvas to get details. For this recitation we will only keep votes from the 2020  presidential election and the total 2020 population counts. You can use additional columns (e.g., Governor's elections results, voting age (VAP) population counts, or the composite Dem/Rep score)

In [ ]:
nj_precinct_data = pd.read_csv('Map_Data/precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','Total_2020_Total','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data

,GEOID20,District,Total_2020_Pres,Dem_2020_Pres,Rep_2020_Pres,Total_2020_Total,White_2020_Total,Hispanic_2020_Total,Black_2020_Total,Asian_2020_Total,Native_2020_Total,Pacific_2020_Total
0,34001005101,2,876,393,472,1240,946,128,102,66,24,0
1,34001005102,2,852,450,388,1913,1331,211,286,84,38,4
2,34001005103,2,1206,517,672,1760,1375,177,78,106,20,2
3,34001005201,2,828,348,469,1311,906,168,150,64,50,5
4,34001005202,2,868,579,282,1892,537,336,598,450,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...
6356,34041115002,7,606,182,418,737,714,11,3,8,0,0
6357,34041115003,7,617,187,418,934,820,60,26,10,14,0
6358,34041115004,7,478,160,308,697,602,66,16,4,5,0
6359,34041115005,7,592,201,381,930,831,47,27,11,12,0


### This is the precinct boundary data (uses shapely)

This is data that represents the geography of the districts. It is needed to test for contiguity, or for any districting partitioning method based on geography. The data is in Shapely format. Each district is represented as a set of points that are connected to create the district shape (in the long/lat coordinates). Shapely geometric functions can be used to compare the shapes. These can be quite inefficient to run, so I am also providing you a pre-computed index that, for each district, lists the districts that are contiguous to it. You can see the code to generate the index in Contiguity.ipynb.

To manipulate the shapes, cast them into Shapely Polygons (see example below) and you can use the Polygon properties and functions: https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon

In [ ]:
shpfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shp'
dbffile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.dbf'
shxfile = 'Map_Data/nj_vtd_2020_bound/nj_vtd_2020_bound.shx'


shpfile = shp.Reader(shp=shpfile, shx=shxfile, dbf=dbffile)
nj_precinct_boundaries={}
for sr in shpfile.iterShapeRecords():
    geom = sr.shape # get geo bit
    rec = sr.record # get db fields
    nj_precinct_boundaries[rec[3]]=geom

### This is the precinct boundary data 

This use the contiguity index I have pre-computed using Contiguity.ipynb, that is stored in Contiguity_nj.csv. 

In [ ]:
nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)

In [ ]:
for i in range(1,13):
    print("District "+str(i)+" "+str(isDistrictContiguous(i, nj_current_assignment, nj_contiguity)))

District 1 True
District 2 True
District 3 True
District 4 True
District 5 True
District 6 True
District 7 True
District 8 True
District 9 True
District 10 True
District 11 True
District 12 True


In [ ]:
#Compactness of the current assignment
for district in range(1,13):
    print("D"+str(district)+" PP : "+str(pp_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    print("D"+str(district)+" BR : "+str(box_reock_compactness(getDistrictShape(district,nj_current_assignment,nj_precinct_boundaries))))
    

D1 PP : 0.41768102211569347
D1 BR : 0.45075813446417157
D2 PP : 0.2632665176502347
D2 BR : 0.38278056561756263
D3 PP : 0.2280937682959879
D3 BR : 0.38134920642809134
D4 PP : 0.24812480573284196
D4 BR : 0.5390173747196018
D5 PP : 0.2410116694999733
D5 BR : 0.36320426268176653
D6 PP : 0.14677124653853732
D6 BR : 0.32853496486220907
D7 PP : 0.20246375771704353
D7 BR : 0.44049249082841035
D8 PP : 0.11227347882175574
D8 BR : 0.36670629634952656
D9 PP : 0.1683197710884751
D9 BR : 0.29705227593212374
D10 PP : 0.12061263370064262
D10 BR : 0.34528827672774703
D11 PP : 0.22236600778446886
D11 BR : 0.5557086439792166
D12 PP : 0.1620092442171186
D12 BR : 0.38520439164401626


In [ ]:
# District Population of the current assignment
print(getDistrictPopulations(nj_current_assignment,nj_precinct_data, 12))

{1: 775340, 2: 778354, 3: 778489, 4: 767834, 5: 774454, 6: 778516, 7: 785173, 8: 800074, 9: 766863, 10: 746178, 11: 769523, 12: 768196}


# Now create your own redistricting maps
Remember to check for contiguity, and to ensure that the population of the districts are balanced (which is not the case in the example above.)

In [ ]:
# --- 1. PREP YOUR DATA FOR SPEED ---

import pandas as pd
import numpy as np
import ast
import random
import math
import networkx as nx

# --- Convert existing DataFrames into fast Dictionaries ---
# (Assumes 'nj_current_assignment', 'nj_contiguity', 'nj_precinct_data',
#  and 'nj_precinct_boundaries' are loaded from the cells above)

# Convert the assignment DataFrame to a dictionary: {GEOID: District}
# This is your main "map" object
current_map = pd.Series(
    nj_current_assignment['District'].values, 
    index=nj_current_assignment['GEOID20']
).to_dict()

# Convert contiguity DataFrame to a dictionary: {GEOID: {Neighbor1, Neighbor2, ...}}
adj_list = {}
for index, row in nj_contiguity.iterrows():
    # row[0] is the Precinct GEOID, row[1] is the string-list of neighbors
    adj_list[row[0]] = set(ast.literal_eval(row[1]))

# Convert precinct data DataFrame to a dictionary: {GEOID: {col: val, ...}}
precinct_data = nj_precinct_data.set_index('GEOID20').to_dict('index')

# Store the shapefile boundaries (already loaded)
boundaries = nj_precinct_boundaries

# --- Calculate Population Targets ---
total_pop = nj_precinct_data['Total_2020_Total'].sum()
target_pop = total_pop / 12
print(f"Total Population: {total_pop}")
print(f"Target Population per District: {target_pop:.2f}")

# --- Keep copies of the original DataFrames for the GIVEN helper functions ---
# We need these *once* to get the initial population
nj_precinct_data_df = nj_precinct_data.copy()
nj_contiguity_df = nj_contiguity.copy()

print("Data prep complete. Fast dictionaries are built.")

/tmp/ipykernel_2417956/2269566849.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  adj_list[row[0]] = set(ast.literal_eval(row[1]))


Total Population: 9288994
Target Population per District: 774082.83
Data prep complete. Fast dictionaries are built.


In [ ]:
# --- 2. YOUR NEW, FAST HELPER FUNCTIONS ---

# Finds all precincts on the border of two different districts
def get_border_precincts(current_map_dict, adj_list_dict):
    border_precincts = []
    for precinct_id, district_id in current_map_dict.items():
        my_district = district_id
        # Use .get() for safety, in case a precinct has no neighbors
        for neighbor_id in adj_list_dict.get(precinct_id, set()):
            # Check if neighbor is in the map
            if neighbor_id in current_map_dict:
                # If a neighbor is in a DIFFERENT district, this is a border precinct
                if current_map_dict[neighbor_id] != my_district:
                    border_precincts.append(precinct_id)
                    break # Stop checking neighbors, we know it's a border
    return border_precincts

# A blazing-fast population check that just reads a 12-item dictionary
def check_balance_from_dict(district_pops, target_pop, tolerance=0.05):
    # This tolerance (0.05 = 5%) is from the assignment
    for pop in district_pops.values():
        if not (target_pop * (1 - tolerance) < pop < target_pop * (1 + tolerance)):
            return False
    return True

# A helper to get vote totals for one district
def get_district_vote_totals(district_id, current_map_dict, precinct_data_dict):
    dem_votes = 0
    rep_votes = 0
    for precinct_id, d_id in current_map_dict.items():
        if d_id == district_id:
            dem_votes += precinct_data_dict[precinct_id].get('Dem_2020_Pres', 0)
            rep_votes += precinct_data_dict[precinct_id].get('Rep_2020_Pres', 0)
    return dem_votes, rep_votes

# Helper to get the total number of seats won
def get_total_seats(current_map_dict, precinct_data_dict):
    total_seats_won = 0
    # Assumes districts are 1-12
    for i in range(1, 13):
        dem_votes, rep_votes = get_district_vote_totals(i, current_map_dict, precinct_data_dict)
        if dem_votes > rep_votes:
            total_seats_won += 1
    return total_seats_won

# A new, FAST contiguity check that uses our dictionaries
def is_contiguous_fast(district_id, current_map_dict, adj_list_dict):
    # Find all precincts in this district
    nodes_in_district = [
        precinct_id for precinct_id, d_id in current_map_dict.items() 
        if d_id == district_id
    ]
    if not nodes_in_district:
        return True # An empty district is "contiguous"

    # Build a graph *only* for these nodes
    district_graph = nx.Graph()
    district_graph.add_nodes_from(nodes_in_district)
    
    start_node = nodes_in_district[0] # Pick one node to start traversal
    
    for precinct_id in nodes_in_district:
        # Look at neighbors (fast lookup)
        for neighbor in adj_list_dict.get(precinct_id, set()):
            # If the neighbor is ALSO in the district, add an edge
            if neighbor in current_map_dict and current_map_dict[neighbor] == district_id:
                district_graph.add_edge(precinct_id, neighbor)
                
    # Check if the graph is connected
    return nx.is_connected(district_graph)

print("Fast helper functions are defined.")

Fast helper functions are defined.


In [ ]:
# --- 3. THE FINAL OPTIMIZED LOOP (FAIR MAP - COMPACTNESS) ---

# --- 1. SETUP ---
map_to_optimize = current_map.copy()

# Get initial population state
df_map_for_pop = pd.DataFrame(map_to_optimize.items(), columns=['GEOID20', 'District'])
district_populations = getDistrictPopulations(df_map_for_pop, nj_precinct_data_df, 12)

current_cut_edges = get_cut_edges(map_to_optimize, adj_list)

# Since the logic below seeks a HIGHER score, we make the score negative.
# We weight compactness heavily (x10) vs population balance.
current_pop_score = 1.0 / np.std(list(district_populations.values()))
current_score = (-1 * current_cut_edges * 10) + current_pop_score

print(f"Starting FAIR optimization. Initial Cut Edges: {current_cut_edges} (Score: {current_score})")

# --- 2. MAIN LOOP ---
ITERATIONS = 50000 

for i in range(ITERATIONS):
    if i % 5000 == 0 and i > 0:
        print(f"Iteration {i}, Cut Edges: {current_cut_edges}, Score: {current_score}")

    # 1. Get a random precinct to flip (Same as before)
    border_precincts = get_border_precincts(map_to_optimize, adj_list)
    if not border_precincts: continue
    precinct_to_flip = random.choice(border_precincts)
    old_district = map_to_optimize[precinct_to_flip]
    
    # 2. Find a valid district to flip into (Same as before)
    possible_new_districts = []
    for neighbor in adj_list[precinct_to_flip]:
        if neighbor in map_to_optimize and map_to_optimize[neighbor] != old_district:
            possible_new_districts.append(map_to_optimize[neighbor])
    if not possible_new_districts: continue
    new_district = random.choice(possible_new_districts)

    # --- 3. UPDATE STATE (Same as before) ---
    pop_to_move = precinct_data[precinct_to_flip]['Total_2020_Total']
    district_populations[old_district] -= pop_to_move
    district_populations[new_district] += pop_to_move
    map_to_optimize[precinct_to_flip] = new_district
    
    constraints_met = (
        check_balance_from_dict(district_populations, target_pop) and
        is_contiguous_fast(old_district, map_to_optimize, adj_list) and
        is_contiguous_fast(new_district, map_to_optimize, adj_list)
    )

    # --- 4. EVALUATE THE MOVE (CHANGED) ---
    if constraints_met:
        # Calculate new compactness
        new_cut_edges = get_cut_edges(map_to_optimize, adj_list)
        new_pop_score = 1.0 / np.std(list(district_populations.values()))
        
        # NEW SCORING FORMULA
        new_score = (-1 * new_cut_edges * 10) + new_pop_score

        # Simulated Annealing Logic
        move_kept = False
        if new_score > current_score:
            move_kept = True 
        else:
            # Temperature calculation
            temperature = 1.0 / (i + 1) 
            # We scale the difference to avoid overflow errors in exp
            diff = new_score - current_score
            try:
                acceptance_prob = math.exp(diff / temperature)
            except OverflowError:
                acceptance_prob = 0
            
            if random.random() < acceptance_prob:
                move_kept = True

        if move_kept:
            current_score = new_score
            current_cut_edges = new_cut_edges
        else:
            # Revert
            map_to_optimize[precinct_to_flip] = old_district
            district_populations[old_district] += pop_to_move
            district_populations[new_district] -= pop_to_move
            
    else:
        # Revert
        map_to_optimize[precinct_to_flip] = old_district
        district_populations[old_district] += pop_to_move
        district_populations[new_district] -= pop_to_move

# --- 5. FINISH ---
print(f"Optimization Complete! Final Cut Edges: {current_cut_edges}")

final_map_df = pd.DataFrame(map_to_optimize.items(), columns=['GEOID20', 'District'])
final_map_df.to_csv('fair_map.csv', index=False)
print(f"Saved map to fair_map.csv")

Starting UNFAIR optimization. Initial Score: 10000.0001 (10 seats)
Iteration 5000, Score: 10000.000040918121 (10 seats)
Iteration 10000, Score: 10000.000038221708 (10 seats)
Iteration 15000, Score: 10000.000039455575 (10 seats)
Iteration 20000, Score: 10000.000037376127 (10 seats)


In [10]:
# --- Code to check "percent different" ---

# 1. Load the original 2022 map (this was loaded in your notebook)
# Make sure the variable 'nj_current_assignment' is still in memory
original_map = nj_current_assignment[['GEOID20', 'District']]
original_map = original_map.rename(columns={'District': 'District_Original'})

# 2. Load your new map
new_map = pd.read_csv('unfair_map.csv')
new_map = new_map[['GEOID20', 'District']]
new_map = new_map.rename(columns={'District': 'District_New'})

# 3. Merge them together on the precinct ID
comparison_df = pd.merge(original_map, new_map, on='GEOID20')

# 4. Count how many precincts are in a different district
different_precincts = (comparison_df['District_Original'] != comparison_df['District_New']).sum()

# 5. Get the total number of precincts
total_precincts = len(comparison_df)

# 6. Calculate the percentage
percent_different = (different_precincts / total_precincts) * 100

print(f"Total precincts: {total_precincts}")
print(f"Precincts assigned to a new district: {different_precincts}")
print(f"Your map is {percent_different:.2f}% different from the 2022 map.")

NameError: name 'nj_current_assignment' is not defined